# Today's cryptocurrencies historical data on by Market Cap 

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import pendulum
from sqlalchemy import create_engine
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
from splinter import Browser
from bs4 import BeautifulSoup
import requests
from config import api_key, password
import urllib.request
from fs.osfs import OSFS

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\jich-\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


#### Context : Finance

#### Goal: Our goal is to create tables showing today’s 10 best/most profitable cryptocurrencies to invest in. We will look at historical data for the previous year and create a database by hourly and daily price changes. 

#### METHOD: we aim to do this by extracting data based on the ranks of the cryptocurrency and also the daily and hourly change of the cryptocurrency. 

#### FINDING DATA: Following are the sources for finding and collecting our data. Sources: Cryptocurrencies: https://coinmarketcap.com/ 
CSV files: https://www.cryptodatadownload.com/data/binance/
APIs: https://financialmodelingprep.com/api/v3/ https://rest.coinapi.io/v1/

#### Data collection tools: Splinter APIs BeautifulSoup

#### DATA AND CLEANUP: 
##### Cleanup process: Pandas SqlAlchemy Database: PostgreSQL

##### TEAM MEMBERS Juan Castaneda Elif Evrim Polat Nichole Edet

<div><h3 style="color:green;">Web Scraping - Obtaining today's best cryptocurrencies liste at https://coinmarketcap.com </h3></div>

In [3]:
# Obtaining Tables at the next link using pandas
url = 'https://coinmarketcap.com/'

In [4]:
tables = pd.read_html(url)

In [5]:
df = tables[0]
df.head()

,Unnamed: 0,#,Name,Price,24h %,7d %,Market Cap,Volume(24h),Circulating Supply,Last 7 Days,Unnamed: 10
0,NaN,1.0,Bitcoin1BTCBuy,"$61,634.42",1.86%,5.31%,"$1,151,601,695,770","$84,719,859,0741,374,554 BTC","18,684,393 BTC",NaN,NaN
1,NaN,2.0,Ethereum2ETHBuy,"$2,413.08",1.96%,16.00%,"$278,700,072,921","$37,316,362,37115,464,190 ETH","115,495,469 ETH",NaN,NaN
2,NaN,3.0,Binance Coin3BNBBuy,$514.51,6.95%,15.28%,"$78,942,616,808","$6,929,470,88413,468,122 BNB","153,432,897 BNB",NaN,NaN
3,NaN,4.0,XRP4XRP,$1.62,7.87%,57.73%,"$73,476,539,590","$21,098,301,65213,037,466,078 XRP","45,404,028,640 XRP",NaN,NaN
4,NaN,5.0,Tether5USDTBuy,$1.00,0.01%,0.15%,"$46,988,484,417","$225,427,683,860224,866,028,798 USDT","46,871,412,194 USDT",NaN,NaN


In [6]:
# using only the tables that will match our PostgreSQL database schema
columns = ['#', 'Name', 'Price']
cryptos_df = df[columns]
cryptos_df.head()

,#,Name,Price
0,1.0,Bitcoin1BTCBuy,"$61,634.42"
1,2.0,Ethereum2ETHBuy,"$2,413.08"
2,3.0,Binance Coin3BNBBuy,$514.51
3,4.0,XRP4XRP,$1.62
4,5.0,Tether5USDTBuy,$1.00


In [7]:
# converting # column values to integer
cryptos_df['#'] = cryptos_df['#'].astype(float).map("{:.0f}".format).copy()
cryptos_df.rename(columns={'#':'id','Name':'coin_name','Price':'latest_price'}, inplace=True)
# obtaiting only the first 10 coins
cryptos_df = cryptos_df[:10]

In [8]:
cryptos_df['latest_price'] = cryptos_df['latest_price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
cryptos_df.head()

,id,coin_name,latest_price
0,1,Bitcoin1BTCBuy,61634.42
1,2,Ethereum2ETHBuy,2413.08
2,3,Binance Coin3BNBBuy,514.51
3,4,XRP4XRP,1.62
4,5,Tether5USDTBuy,1.00


In [9]:
# Splitting Name column into 2 different columns which will contain the coin name and coin symbol using a number as a delimiter
# and using n=1 to do this only once, in case the coin name contains a number already
columnsplit = cryptos_df['coin_name'].str.split('(\d+)',n=1, expand=True)
columnsplit.head()

,0,1,2
0,Bitcoin,1,BTCBuy
1,Ethereum,2,ETHBuy
2,Binance Coin,3,BNBBuy
3,XRP,4,XRP
4,Tether,5,USDTBuy


###### Formatting table to match our postgreSQL table in our DB

In [10]:
columnsplit = cryptos_df['coin_name'].str.split('(\d+)',n=1, expand=True)
# since coin_symbol column originally contains a recommendation to "Buy" if applicable on coinmarketcap, we will get rid of that
# columnsplit[1] = columnsplit[1].str.replace('Buy', '', regex=True) by ignoring index 1
cryptos_df = cryptos_df.assign(coin_name=columnsplit[0],coin_symbol=columnsplit[2].str.replace('Buy',''))

In [11]:
cryptos_df

,id,coin_name,latest_price,coin_symbol
0,1,Bitcoin,61634.4200,BTC
1,2,Ethereum,2413.0800,ETH
2,3,Binance Coin,514.5100,BNB
3,4,XRP,1.6200,XRP
4,5,Tether,1.0000,USDT
5,6,Cardano,1.4200,ADA
6,7,Dogecoin,0.3123,DOGE
7,8,Polkadot,41.4800,DOT
8,9,Litecoin,304.3300,LTC
9,10,Bitcoin Cash,1043.6400,BCH


In [12]:
# rearranging columns
columns = cryptos_df.columns.to_list()

In [13]:
# new order
columns = columns[:2] + [columns[-1]] + [columns[-2]]
columns

['id', 'coin_name', 'coin_symbol', 'latest_price']

In [14]:
cryptos_df = cryptos_df[columns]
cryptos_df

,id,coin_name,coin_symbol,latest_price
0,1,Bitcoin,BTC,61634.4200
1,2,Ethereum,ETH,2413.0800
2,3,Binance Coin,BNB,514.5100
3,4,XRP,XRP,1.6200
4,5,Tether,USDT,1.0000
5,6,Cardano,ADA,1.4200
6,7,Dogecoin,DOGE,0.3123
7,8,Polkadot,DOT,41.4800
8,9,Litecoin,LTC,304.3300
9,10,Bitcoin Cash,BCH,1043.6400


<h3 style="color:orange;"> putting cryptos_df into coins table in pgadmin server</h3>

<h4 style="color:pink;"> Connecting to DB</h4>

In [15]:
connection_string = f"postgres:{password}@localhost:5433/cryptos_db"
engine = create_engine(f'postgresql://{connection_string}')

In [16]:
engine.table_names()

['daily_Price', 'coins', 'daily_price', 'hourly_price']

In [17]:
cryptos_df.to_sql(name='coins', con=engine, if_exists='append', index=False)

In [18]:
# checking for changes in pgadmin cryoptos_db
pd.read_sql_query('select * from coins', con=engine)

,id,coin_name,coin_symbol,latest_price
0,1,Bitcoin,BTC,61634.4200
1,2,Ethereum,ETH,2413.0800
2,3,Binance Coin,BNB,514.5100
3,4,XRP,XRP,1.6200
4,5,Tether,USDT,1.0000
5,6,Cardano,ADA,1.4200
6,7,Dogecoin,DOGE,0.3123
7,8,Polkadot,DOT,41.4800
8,9,Litecoin,LTC,304.3300
9,10,Bitcoin Cash,BCH,1043.6400


<div><h3 style="color:green;">Dowloading the CSV files if available at https://www.cryptodatadownload.com/data/binance/</h3></div>

In [19]:
# Obtaining symbols in the dataframe and storing in a list
symbols = cryptos_df['coin_symbol'].to_list()
symbols

['BTC', 'ETH', 'BNB', 'XRP', 'USDT', 'ADA', 'DOGE', 'DOT', 'LTC', 'BCH']

###### Web Scraping using beautiulsoup

In [20]:
# URL of page to be scraped
url = 'https://www.cryptodatadownload.com/data/binance/'
# instantiating the webdriver for Chrome!!!
browser.visit(url)
# Getting the webpage content
html = browser.html
# parsing our html plain text to a BS object
soup = BeautifulSoup(html, 'html.parser')
browser.quit()

In [21]:
prhs = soup.find_all('p')

In [22]:
bnb_cryptos = prhs[3]
links = bnb_cryptos.find_all('a')
complete_links = ["https://www.cryptodatadownload.com" + link['href'] for link in links if "cdd" in link['href']]

In [23]:
complete_links

['https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_minute.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_minute.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_minute.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_NEOUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_NEOUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_NEOUSDT_minute.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_d.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_1h.csv',
 'https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_minute.c

In [24]:
len(complete_links)

51

In [25]:
with OSFS("./Resources") as myfs:
    if(not myfs.exists("datasets")):
        myfs.makedir("datasets")
        print("Created datasets folder")

In [26]:
crypto_data_found = []
for symbol in (symbols):
    for index,link in enumerate(complete_links):
        if symbol in link:
            if symbol == "USDT":
                crypto_data_found.append((symbol,False))
            else:
                crypto_data_found.append((symbol,True))
            break
        if index==len(complete_links)-1:
            crypto_data_found.append((symbol,False))

In [27]:
#when true it means the file is on https://www.cryptodatadownload.com/cdd/Binance, otherwise we have to use the API
crypto_data_found

[('BTC', True),
 ('ETH', True),
 ('BNB', True),
 ('XRP', True),
 ('USDT', False),
 ('ADA', True),
 ('DOGE', False),
 ('DOT', False),
 ('LTC', True),
 ('BCH', False)]

###### coinAPI setup and use

In [28]:
def get_historic_data(symbol,time_start,time_end,period="1DAY",limit=10000):
    url = f"https://rest.coinapi.io/v1/ohlcv/{symbol}/USD/history?period_id={period}&limit={limit}&time_start={time_start}&time_end={time_end}"
    headers = {"X-CoinAPI-Key" : api_key}
    response = requests.get(url, headers = headers)

    if(response.status_code == 429):
        # API responses exhausted
        return "Too many requests."
    historical_data = response.json()
    print("getting data....")
    try:
        temp_df = pd.DataFrame(historical_data)
        if period == "1DAY":
            temp_df.to_csv(f"./Resources/datasets/{symbol}USDT_d.csv", index=False)
            print(f"{symbol}exported")
        elif period == "1HRS":
            temp_df.to_csv(f"./Resources/datasets/{symbol}USDT_1h.csv", index=False)
            print("exported")
    except Exception as e:
        print(e)
    

###### Obaiting CSV files

In [29]:
for symbol,found in crypto_data_found:
    if found:
        try:
            file_name = f"{symbol}USDT_d.csv"
            daily_records = f"https://www.cryptodatadownload.com/cdd/Binance_{symbol}USDT_d.csv"
            urllib.request.urlretrieve(daily_records, f"./Resources/datasets/{file_name}")
            print(daily_records,"Downloaded")
            file_name = f"{symbol}USDT_1h.csv"
            hourly_records = f"https://www.cryptodatadownload.com/cdd/Binance_{symbol}USDT_1h.csv"
            file_name = f"{symbol}USDT_1h.csv"
            urllib.request.urlretrieve(hourly_records, f"./Resources/datasets/{file_name}")
            print(hourly_records,"Downloaded")
        except Exception as e:
            print(e)
    else:
        try:
            print(f"exporting {symbol}")
            today = pendulum.now().format("YYYY-MM-DDTHH:mm:ss")
            year_ago= pendulum.now().subtract(years=1).format("YYYY-MM-DDTHH:mm:ss")
            get_historic_data(symbol,year_ago,today,period="1DAY")
            get_historic_data(symbol,year_ago,today,period="1HRS")
        except Exception as e:
            print(e)

https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_1h.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_ETHUSDT_1h.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_BNBUSDT_1h.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_XRPUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_XRPUSDT_1h.csv Downloaded
exporting USDT
https://www.cryptodatadownload.com/cdd/Binance_ADAUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_ADAUSDT_1h.csv Downloaded
exporting DOGE
exporting DOT
https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_d.csv Downloaded
https://www.cryptodatadownload.com/cdd/Binance_LTCUSDT_1h.csv Downloaded
exporting BCH


### Cleaning up CSV files from https://www.cryptodatadownload.com/ EXAMPLE

In [30]:
btc_path = "./Resources/datasets/BTCUSDT_1h.csv"
# using skiprows to ignore first row in CSV file
df = pd.read_csv(btc_path, skiprows=1)

In [31]:
df

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USDT,tradecount
0,1.618531e+12,2021-04-16 00:00:00,BTC/USDT,63158.74,63326.80,63158.73,63222.86,118.809924,7.515525e+06,4401.0
1,1.618528e+12,2021-04-15 23:00:00,BTC/USDT,63252.27,63339.99,62963.78,63159.98,1659.527690,1.048196e+08,61042.0
2,1.618524e+12,2021-04-15 22:00:00,BTC/USDT,63369.51,63518.22,63179.20,63252.27,2054.258481,1.301072e+08,77169.0
3,1.618520e+12,2021-04-15 21:00:00,BTC/USDT,63350.88,63800.00,63226.72,63369.50,2038.260233,1.293121e+08,91003.0
4,1.618517e+12,2021-04-15 20:00:00,BTC/USDT,63363.29,63592.24,63181.37,63350.89,3047.729738,1.932230e+08,119185.0
...,...,...,...,...,...,...,...,...,...,...
34742,1.502957e+09,2017-08-17 08-AM,BTC/USDT,4349.99,4377.85,4333.32,4360.69,0.949900,4.139700e+03,NaN
34743,1.502953e+09,2017-08-17 07-AM,BTC/USDT,4324.35,4349.99,4287.41,4349.99,4.440000,1.924106e+04,NaN
34744,1.502950e+09,2017-08-17 06-AM,BTC/USDT,4315.32,4345.45,4309.37,4324.35,7.230000,3.128231e+04,NaN
34745,1.502946e+09,2017-08-17 05-AM,BTC/USDT,4308.83,4328.69,4291.37,4315.32,23.230000,1.003048e+05,NaN


In [32]:
df['coin_id'] = 1
# getting only the columns that will match our sql table structure
btc_df = df[['coin_id','date','open','high','low','close']]
btc_df

,coin_id,date,open,high,low,close
0,1,2021-04-16 00:00:00,63158.74,63326.80,63158.73,63222.86
1,1,2021-04-15 23:00:00,63252.27,63339.99,62963.78,63159.98
2,1,2021-04-15 22:00:00,63369.51,63518.22,63179.20,63252.27
3,1,2021-04-15 21:00:00,63350.88,63800.00,63226.72,63369.50
4,1,2021-04-15 20:00:00,63363.29,63592.24,63181.37,63350.89
...,...,...,...,...,...,...
34742,1,2017-08-17 08-AM,4349.99,4377.85,4333.32,4360.69
34743,1,2017-08-17 07-AM,4324.35,4349.99,4287.41,4349.99
34744,1,2017-08-17 06-AM,4315.32,4345.45,4309.37,4324.35
34745,1,2017-08-17 05-AM,4308.83,4328.69,4291.37,4315.32


In [33]:
# since some of the dates in the csv file are in the next form "2020-11-20 07-AM", we will need to reformat them so SQL can 
# read it as a Date format, Example: "2020-11-20 07-AM" after formatting would be ""2020-11-20 07:00:00""
def format_date(date):
    try:
        date, hour = date.split(" ")
        if "AM" in hour:
            date = f"{date} {hour.replace('-AM',':00:00')}"
            return date
        elif "PM" in hour:
            date = f"{date} {int(hour.replace('-PM',''))+12}:00:00"
            return date
        else:
            return f"{date} {hour}"
    except:
        return f"{date.strip(' ')} 00:00:00"


In [34]:
btc_df['date']= btc_df['date'].apply(lambda x : format_date(x))
btc_df

,coin_id,date,open,high,low,close
0,1,2021-04-16 00:00:00,63158.74,63326.80,63158.73,63222.86
1,1,2021-04-15 23:00:00,63252.27,63339.99,62963.78,63159.98
2,1,2021-04-15 22:00:00,63369.51,63518.22,63179.20,63252.27
3,1,2021-04-15 21:00:00,63350.88,63800.00,63226.72,63369.50
4,1,2021-04-15 20:00:00,63363.29,63592.24,63181.37,63350.89
...,...,...,...,...,...,...
34742,1,2017-08-17 08:00:00,4349.99,4377.85,4333.32,4360.69
34743,1,2017-08-17 07:00:00,4324.35,4349.99,4287.41,4349.99
34744,1,2017-08-17 06:00:00,4315.32,4345.45,4309.37,4324.35
34745,1,2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32


### Cleaning up CSV files from https://www.coinapi.io/ EXAMPLE

In [35]:
dot_path = "./Resources/datasets/DOTUSDT_d.csv"
df1 = pd.read_csv(dot_path)

In [36]:
df1

,time_period_start,time_period_end,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count
0,2020-08-18T00:00:00.0000000Z,2020-08-19T00:00:00.0000000Z,2020-08-18T17:02:07.9180000Z,2020-08-18T23:59:25.8850000Z,50.0000,50.7000,2.9000,3.1099,3.421890e+06,8241
1,2020-08-19T00:00:00.0000000Z,2020-08-20T00:00:00.0000000Z,2020-08-19T00:00:10.6230000Z,2020-08-19T23:58:50.5120000Z,3.1099,3.1600,2.7100,2.9175,1.440782e+06,7624
2,2020-08-20T00:00:00.0000000Z,2020-08-21T00:00:00.0000000Z,2020-08-20T00:00:04.7500000Z,2020-08-20T23:59:50.5420000Z,2.9175,2.9600,2.7300,2.9090,9.427870e+05,3926
3,2020-08-21T00:00:00.0000000Z,2020-08-22T00:00:00.0000000Z,2020-08-21T00:01:08.8630000Z,2020-08-21T23:59:59.0660000Z,2.9380,3.7000,2.8000,3.0776,1.100527e+06,5994
4,2020-08-22T00:00:00.0000000Z,2020-08-23T00:00:00.0000000Z,2020-08-22T00:00:24.2980000Z,2020-08-22T23:59:57.7560000Z,3.0776,4.4866,2.8500,4.4515,4.042307e+06,17032
...,...,...,...,...,...,...,...,...,...,...
237,2021-04-12T00:00:00.0000000Z,2021-04-13T00:00:00.0000000Z,2021-04-12T00:00:00.0240000Z,2021-04-12T23:39:38.9170000Z,41.2955,41.7190,39.3053,40.5707,4.024652e+05,7913
238,2021-04-13T00:00:00.0000000Z,2021-04-14T00:00:00.0000000Z,2021-04-13T00:00:00.5830000Z,2021-04-13T23:59:45.2330000Z,40.4377,43.3500,39.8104,42.9524,8.086810e+05,14302
239,2021-04-14T00:00:00.0000000Z,2021-04-15T00:00:00.0000000Z,2021-04-14T00:00:00.0020000Z,2021-04-14T22:11:34.5050000Z,42.9228,44.5231,40.1916,42.0791,7.328068e+05,15871
240,2021-04-15T00:00:00.0000000Z,2021-04-16T00:00:00.0000000Z,2021-04-15T00:00:00.0020000Z,2021-04-15T23:59:41.2650000Z,42.4943,44.3791,41.3727,43.4838,7.200537e+05,13938


In [37]:
df1['coin_id'] = 1
odt_df = df1[['coin_id','time_period_start','price_open','price_high','price_low','price_close']]
odt_df.rename(columns={"time_period_start":"date","price_open":"open","price_high":"high","price_low":"low",
                       "price_close":"close"}, inplace=True)
columnsplit = odt_df['date'].str.split(".",n=1, expand=True)
odt_df = odt_df.assign(date=columnsplit[0].str.replace("T"," "))
pd.to_datetime(odt_df['date'], infer_datetime_format=True)
# We need to sort df in descending order(latest date first)
odt_df.sort_values(by=['date'], ascending=False, inplace=True)
odt_df

,coin_id,date,open,high,low,low
241,1,2021-04-16 00:00:00,43.4435,43.8010,40.1408,41.8482
240,1,2021-04-15 00:00:00,42.4943,44.3791,41.3727,43.4838
239,1,2021-04-14 00:00:00,42.9228,44.5231,40.1916,42.0791
238,1,2021-04-13 00:00:00,40.4377,43.3500,39.8104,42.9524
237,1,2021-04-12 00:00:00,41.2955,41.7190,39.3053,40.5707
...,...,...,...,...,...,...
4,1,2020-08-22 00:00:00,3.0776,4.4866,2.8500,4.4515
3,1,2020-08-21 00:00:00,2.9380,3.7000,2.8000,3.0776
2,1,2020-08-20 00:00:00,2.9175,2.9600,2.7300,2.9090
1,1,2020-08-19 00:00:00,3.1099,3.1600,2.7100,2.9175


<div style="background-color:pink;font-size:20px;"><h3 style="color:red;"> We're going to utilize a for loop to automate the clean up process</h3></div>

###### Retrieving table names in our database

In [38]:
# reutilizing our previous engine instance
engine.table_names()

['daily_Price', 'coins', 'daily_price', 'hourly_price']

###### Loading Dataframes into our respective tables

In [39]:
def load_df_to_sql(df, table_name):
    try:
        df.to_sql(name=table_name, con=engine, if_exists='append', index=False)
    except Exception as e:
        print(e)

In [40]:
for crypto,found in crypto_data_found:
    coin_id = cryptos_df.loc[cryptos_df['coin_symbol'].isin([crypto])]['id']
    if found:    # if found= True, it means the data was obtained from  https://www.cryptodatadownload.com/ 
        for i in ['1h','d']:
            df_path = f"./Resources/datasets/{crypto}USDT_{i}.csv"
            # using skiprows to ignore first row in CSV file
            df = pd.read_csv(df_path, skiprows=1)
            df['coin_id'] = int(coin_id)
            df_new = df[['coin_id','date','open','high','low','close']]
            df_new['date']= df_new['date'].apply(lambda x : format_date(x))
            if i == "1h":
                load_df_to_sql(df_new, "hourly_price")
            if i == "d":
                load_df_to_sql(df_new, "daily_price")
                
    if found == False: # the data was obtained from the coinAPI
        for i in ['1h','d']:
            df_path = f"./Resources/datasets/{crypto}USDT_{i}.csv"
            df = pd.read_csv(df_path) 
            df['coin_id'] = int(coin_id)
            df_new = df[['coin_id','time_period_start','price_open','price_high','price_low','price_close']]
            df_new.rename(columns={"time_period_start":"date","price_open":"open","price_high":"high","price_low":"low",
                                   "price_close":"close"}, inplace=True)
            columnsplit = df_new['date'].str.split(".",n=1, expand=True)
            df_new = df_new.assign(date=columnsplit[0].str.replace("T"," "))
            pd.to_datetime(df_new['date'], infer_datetime_format=True)
            # We need to sort df in descending order(latest date first)
            odt_df.sort_values(by=['date'], ascending=False, inplace=True)
            if i == "1h":
                load_df_to_sql(df_new, "hourly_price")
            if i == "d":
                load_df_to_sql(df_new, "daily_price")
         

In [42]:
# checking for changes in pgadmin cryoptos_db
pd.read_sql_query('select * from daily_price', con=engine)

,id,coin_id,date,open,high,low,close
0,1,1,2021-04-16 00:00:00,63158.74,63326.80,63158.73,63229.91
1,2,1,2021-04-15 00:00:00,62959.53,63800.00,62020.00,63159.98
2,3,1,2021-04-14 00:00:00,63575.01,64854.00,61301.00,62959.53
3,4,1,2021-04-13 00:00:00,59860.01,63777.77,59805.15,63575.00
4,5,1,2021-04-12 00:00:00,59998.80,61300.00,59350.59,59860.00
...,...,...,...,...,...,...,...
8624,8625,10,2021-04-12 00:00:00,689.44,696.48,670.86,NaN
8625,8626,10,2021-04-13 00:00:00,670.41,751.97,744.81,NaN
8626,8627,10,2021-04-14 00:00:00,744.90,845.74,815.05,NaN
8627,8628,10,2021-04-15 00:00:00,814.86,879.00,866.30,NaN
